In [2]:
import pandas as pd
import sqlite3

In [3]:
orders = pd.read_csv("orders.csv")
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [4]:
users=pd.read_json("users.json")
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [7]:
import sqlite3
import pandas as pd

conn= sqlite3.connect(":memory")

cursor = conn.cursor()
cursor.execute("Drop table if exists restaurants;")

with open("restaurants.sql") as f:conn.executescript(f.read())

restaurants=pd.read_sql("select * from restaurants",conn)
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [8]:
df=orders.merge(users, on="user_id",how="left")
df=df.merge(restaurants, on="restaurant_id",how='left')
df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [9]:
df.to_csv("final_food_delivery_dataset.csv",index= False)

In [10]:
df[df["membership"]=="Gold"]\
  .groupby("city")["total_amount"]\
  .sum()\
  .sort_values(ascending=False)

,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [11]:
df.groupby("cuisine")["total_amount"]\
  .mean()\
  .sort_values(ascending=False)

,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [21]:
user_total=df.groupby("user_id")["total_amount"].sum()

count_users=user_total[user_total>1000].count()
count_users

np.int64(2544)

In [22]:
df.groupby("restaurant_rating")
["total_amount"]\
  .sum()\
  .sort_values(ascending=False)

KeyError: 'restaurant_rating'

In [23]:
df.columns

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')

In [25]:
df.groupby("rating")["total_amount"]\
  .sum()\
  .sort_values(ascending=False)

,total_amount
rating,
4.8,657707.71
4.6,495867.97
3.2,490913.01
4.5,479047.03
4.9,467467.09
3.8,466878.69
3.1,443863.92
4.2,423185.06
4.7,416301.51


In [26]:
df[df["membership"]=="Gold"]\
  .groupby("city")["total_amount"]\
  .mean()\
  .sort_values(ascending=False)

,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [31]:
df.groupby("cuisine")
["restaurant_name"].nunique()\
  .sort_values()

AttributeError: 'list' object has no attribute 'nunique'

In [33]:
df.groupby("cuisine")["restaurant_name_x"]\
  .nunique()\
  .sort_values()

,restaurant_name_x
cuisine,
Chinese,115
Indian,122
Italian,124
Mexican,124


In [35]:
df.groupby("cuisine")["total_amount"]\
  .sum()\
  .sort_values(ascending=False)

,total_amount
cuisine,
Mexican,2085503.09
Italian,2024203.80
Indian,1971412.58
Chinese,1930504.65


In [36]:
total_orders=len(df)
gold_orders=len(df[df["membership"]=="Gold"])

percentage=(gold_orders / total_orders)*100
round(percentage)

50

In [38]:
df.groupby(["membership","cuisine"])["total_amount"]\
  .sum()\
  .sort_values(ascending=False)

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [41]:
restaurant_stats=df.groupby("restaurant_name_x").agg(
    total_orders=("order_id","count"),
    avg_order_value=("total_amount","mean")
    )
restaurant_stats

,total_orders,avg_order_value
restaurant_name_x,,
Amma Biryani Family Restaurant,45,799.941556
Amma Biryani North Indian,57,774.900526
Amma Biryani South Indian,15,708.708667
Amma Cafe Non-Veg,24,736.174583
Amma Cafe North Indian,11,667.979091
...,...,...
Udupi Mess Pure Veg,22,768.489091
Udupi Restaurant Family Restaurant,19,796.961053
Udupi Restaurant Pure Veg,10,895.619000


In [42]:
restaurant_stats[restaurant_stats["total_orders"]<20]\
  .sort_values("avg_order_value",ascending=False)

,total_orders,avg_order_value
restaurant_name_x,,
Hotel Dhaba Multicuisine,13,1040.222308
Sri Mess Punjabi,12,1029.180833
Ruchi Biryani Punjabi,16,1002.140625
Sri Delights Pure Veg,18,989.467222
Classic Kitchen Family Restaurant,19,973.167895
...,...,...
Annapurna Tiffins Punjabi,19,621.828947
Darbar Tiffins Non-Veg,18,596.815556
Darbar Restaurant Punjabi,14,589.972857


In [43]:
# ensure order_date is datetime
df["order_date"] = pd.to_datetime(df["order_date"])

# create quarter
df["quarter"] = df["order_date"].dt.to_period("Q")

# total revenue per quarter
quarter_revenue = df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False)
quarter_revenue

/tmp/ipython-input-178733016.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["order_date"] = pd.to_datetime(df["order_date"])


,total_amount
quarter,
2023Q3,2037385.10
2023Q4,2018263.66
2023Q1,1993425.14
2023Q2,1945348.72
2024Q1,17201.50


In [44]:
df[df["membership"] == "Gold"].shape[0]

4987

In [45]:
round(df[df["city"] == "Hyderabad"]["total_amount"].sum())

1889367

In [46]:
df["user_id"].nunique()

2883

In [47]:
round(
    df[df["membership"] == "Gold"]["total_amount"].mean(),
    2
)

np.float64(797.15)

In [48]:
df[df["rating"] >= 4.5].shape[0]

3374

In [49]:
gold_city_revenue = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

gold_city_revenue

,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [50]:
df[(df["membership"] == "Gold") & (df["city"] == "Chennai")].shape[0]


1337

In [51]:
df.shape[0]

10000